In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install treefarms
!pip install timbertrek

In [2]:
import pandas as pd
import numpy as np
import pathlib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from treefarms.model.threshold_guess import compute_thresholds, cut
from treefarms import TREEFARMS
from treefarms.model.model_set import ModelSetContainer
import timbertrek

### Example from Tutorial
Run treefarms on COMPAS, a recidivism dataset containing 6907 samples and 7 continous features.  Then visualize the Rashomon set using timbertrek package, as well as obtain individual trees from the Rashomon set.

In [3]:
# read the dataset
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/rashomon_treefarms/experiments/datasets/compas/binned.csv')
X, y = df.iloc[:, :-1], df.iloc[:, -1]
h = df.columns[:-1]
df

sex:Female  age:<21  age:<23  age:<26  age:<46  juvenile-felonies:=0  \
0              0        0        0        0        0                     1   
1              0        0        0        0        1                     1   
2              0        0        1        1        1                     1   
3              0        0        0        0        1                     1   
4              0        0        0        0        1                     1   
...          ...      ...      ...      ...      ...                   ...   
6902           0        0        1        1        1                     1   
6903           0        0        1        1        1                     1   
6904           0        0        0        0        0                     1   
6905           1        0        0        0        1                     1   
6906           1        0        1        1        1                     1   

      juvenile-misdemeanors:=0  juvenile-crimes:=0  priors:=0  priors:=1  \
0                            1                   1          1          0   
1                            1                   1          1          0   
2                            1                   0          0          0   
3                            1                   1          1          0   
4                            1                   1          0          0   
...                        ...                 ...        ...        ...   
6902                         1                   1          1          0   
6903                         1                   1          1          0   
6904                         1                   1          1          0   
6905                         1                   1          0          0   
6906                         1                   1          0          0   

      priors:2-3  priors:>3  recidivate-within-two-years:1  
0              0          0                              0  
1              0          0                              1  
2              0          1                              1  
3              0          0                              0  
4              0          1                              1  
...          ...        ...                            ...  
6902           0          0                              0  
6903           0          0                              0  
6904           0          0                              0  
6905           1          0                              0  
6906           1          0                              1  

[6907 rows x 13 columns]

In [4]:
# train TREEFARMS model
config = {
    "regularization": 0.01, # regularization penalizes the tree with more leaves. Recommended to set it to a relative high value to find a sparse tree
    "rashomon_bound_multiplier": 0.05, # rashomon bound multiplier indicates how large of a Rashomon set would you like to get
}

model = TREEFARMS(config)

model.fit(X, y)

treefarms reported successful execution
training completed. Number of trees in the Rashomon set: 1422


We then visualize the Rashomon set

In [5]:
model.visualize()

Generating decision paths from 1422 trees: 100%|██████████| 1422/1422 [00:10<00:00, 132.31it/s]


<iframe
 srcdoc="<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8" /><meta name="viewport" content="width=device-width, initial-scale=1.0" /><title>TimberTrek</title><style>html{font-size:16px;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;text-rendering:optimizeLegibility;-webkit-text-size-adjust:100%;-moz-text-size-adjust:100%}html,body{position:relative;width:100%;height:100%}body{margin:0;padding:0;box-sizing:border-box;font-family:-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Oxygen-Sans,Ubuntu,Cantarell,Helvetica Neue,sans-serif;color:#4a4a4a;font-size:1em;font-weight:400;line-height:1.5}*,:after,:before{box-sizing:inherit}a{color:#0064c8;text-decoration:none}a:hover{text-decoration:underline}a:visited{color:#0050a0}label{display:block}input,button,select,textarea{font-family:inherit;font-size:inherit;-webkit-padding:.4em 0;padding:.4em;margin:0 0 .5em;box-sizing:border-box;border:1px solid #ccc;border-radius:2px}input:disabled{color:#ccc}button{color:#333;background-color:#f4f4f4;outline:none}button:disabled{color:#999}button:not(:disabled):active{background-color:#ddd}button:focus{border-color:#666}</style><script defer src='...'></script><script defer src='...'></script></head><body></body></html>"
 frameBorder="0"
 width="100%"
 height="650px"
 id="timbertrek-iframe-61490253">

In [6]:
first_tree = model[0]
print(f'The accuracy of the first tree on the data is: {first_tree.score(X, y)}')
print(model[0])

The accuracy of the first tree on the data is: 0.6671492688576807
if feature_11 = true then:
    predicted Prediction: 1

else if feature_0 = true and feature_11 != true then:
    predicted Prediction: 0

else if feature_0 != true and feature_10 = true and feature_11 != true and feature_3 = true then:
    predicted Prediction: 1

else if feature_0 != true and feature_10 != true and feature_11 != true and feature_3 = true and feature_7 = true then:
    predicted Prediction: 0

else if feature_0 != true and feature_10 != true and feature_11 != true and feature_3 = true and feature_7 != true then:
    predicted Prediction: 1

else if feature_0 != true and feature_11 != true and feature_3 != true then:
    predicted Prediction: 0


In [7]:
df2 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/rashomon_treefarms/experiments/datasets/car_evaluation/data.csv')
df2

a1_high  a1_low  a1_med  a2_high  a2_low  a2_med  a3_2  a3_3  a3_4  \
0           0       0       0        0       0       0     1     0     0   
1           0       0       0        0       0       0     1     0     0   
2           0       0       0        0       0       0     1     0     0   
3           0       0       0        0       0       0     1     0     0   
4           0       0       0        0       0       0     1     0     0   
...       ...     ...     ...      ...     ...     ...   ...   ...   ...   
1723        0       1       0        0       1       0     0     0     0   
1724        0       1       0        0       1       0     0     0     0   
1725        0       1       0        0       1       0     0     0     0   
1726        0       1       0        0       1       0     0     0     0   
1727        0       1       0        0       1       0     0     0     0   

      a4_2  a4_4  a5_big  a5_med  a6_high  a6_low  class_1  
0        1     0       0       0        0       1        0  
1        1     0       0       0        0       0        0  
2        1     0       0       0        1       0        0  
3        1     0       0       1        0       1        0  
4        1     0       0       1        0       0        0  
...    ...   ...     ...     ...      ...     ...      ...  
1723     0     0       0       1        0       0        1  
1724     0     0       0       1        1       0        1  
1725     0     0       1       0        0       1        0  
1726     0     0       1       0        0       0        1  
1727     0     0       1       0        1       0        1  

[1728 rows x 16 columns]

In [8]:
X, y = df2.iloc[:, :-1], df2.iloc[:, -1]
h = df2.columns[:-1]

In [9]:
# train TREEFARMS model
config = {
    "regularization": 0.01, # regularization penalizes the tree with more leaves. Recommended to set it to a relative high value to find a sparse tree
    "rashomon_bound_multiplier": 0.05, # rashomon bound multiplier indicates how large of a Rashomon set would you like to get
}

model = TREEFARMS(config)

model.fit(X, y)

treefarms reported successful execution
training completed. Number of trees in the Rashomon set: 208


In [10]:
model.visualize()

Generating decision paths from 208 trees: 100%|██████████| 208/208 [00:00<00:00, 415.62it/s]


<iframe
 srcdoc="<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8" /><meta name="viewport" content="width=device-width, initial-scale=1.0" /><title>TimberTrek</title><style>html{font-size:16px;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;text-rendering:optimizeLegibility;-webkit-text-size-adjust:100%;-moz-text-size-adjust:100%}html,body{position:relative;width:100%;height:100%}body{margin:0;padding:0;box-sizing:border-box;font-family:-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Oxygen-Sans,Ubuntu,Cantarell,Helvetica Neue,sans-serif;color:#4a4a4a;font-size:1em;font-weight:400;line-height:1.5}*,:after,:before{box-sizing:inherit}a{color:#0064c8;text-decoration:none}a:hover{text-decoration:underline}a:visited{color:#0050a0}label{display:block}input,button,select,textarea{font-family:inherit;font-size:inherit;-webkit-padding:.4em 0;padding:.4em;margin:0 0 .5em;box-sizing:border-box;border:1px solid #ccc;border-radius:2px}input:disabled{color:#ccc}button{color:#333;background-color:#f4f4f4;outline:none}button:disabled{color:#999}button:not(:disabled):active{background-color:#ddd}button:focus{border-color:#666}</style><script defer src='...'></script><script defer src='...'></script></head><body></body></html>"
 frameBorder="0"
 width="100%"
 height="650px"
 id="timbertrek-iframe-46766867">